# Tinanic

In [1]:
import pandas as pd

In [2]:
BENCH_MARK = .5
DIM = 30
HIDDEN = 512
train_df = pd.read_csv("/data/titanic/Data/train.csv")
test_df = pd.read_csv("/data/titanic/Data/test.csv")
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
train_df["AgeZero"] = train_df["Age"].apply(lambda x:(int(x==0))*1)
test_df["AgeZero"] = test_df["Age"].apply(lambda x:(int(x==0))*1)

In [3]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeZero
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S,0
5,6,0,3,"Moran, Mr. James",male,0.0,0,0,330877,8.4583,0,Q,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,0,S,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,0,S,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,0,C,0


In [4]:
from ray.b4tab import categorical,minmax,tabulate,categorical_idx

In [5]:
Pclass = categorical_idx("Pclass",)
Pclass.build(train_df.Pclass)

   Pclass
3     491
1     216
2     184


In [6]:
Sex = categorical_idx("Sex")
Sex.build(train_df.Sex)

        Sex
male    577
female  314


In [7]:
Embarked = categorical_idx("Embarked")
Embarked.build(train_df.Embarked)

   Embarked
S       644
C       168
Q        77
0         2


In [8]:
AgeZero = categorical_idx("AgeZero")
AgeZero.build(train_df.AgeZero)

   AgeZero
0      714
1      177


In [9]:
Age = minmax("Age")
Age.build(train_df.Age)

min_:0.000 	max_:80.000	 range:80.000


In [10]:
SibSp = minmax("SibSp")
SibSp.build(train_df.SibSp)

min_:0.000 	max_:8.000	 range:8.000


In [11]:
Parch = minmax("Parch")
Parch.build(train_df.Parch)

min_:0.000 	max_:6.000	 range:6.000


In [12]:
Fare = minmax("Fare")
Fare.build(train_df.Fare)

min_:0.000 	max_:512.329	 range:512.329


In [13]:
x_pre = tabulate("x_pre")
x_pre.build(Pclass,Sex,Embarked,AgeZero,Age,SibSp,Parch,Fare)

In [14]:
len(Pclass.cate_list)

3

In [15]:
Survived = minmax("Survived")
Survived.build(train_df["Survived"])

min_:0.000 	max_:1.000	 range:1.000


In [16]:
y_pre = tabulate("y_pre")
y_pre.build(Survived)

In [17]:
# x_pre.prepro(train_df)[:,1:2]

In [18]:
x_pre.prepro(train_df).shape,y_pre.prepro(train_df).shape

((891, 8), (891, 1))

In [19]:
import torch
from torch import nn

In [20]:
class ti_mlp(nn.Module):
    def __init__(self):
        super(ti_mlp,self).__init__()
        self.emb_p = nn.Embedding(len(Pclass.cate_list),DIM)
        self.emb_s = nn.Embedding(len(Sex.cate_list),DIM)
        self.emb_e = nn.Embedding(len(Embarked.cate_list),DIM)
        self.emb_a = nn.Embedding(2,DIM)
        self.mlp = nn.Sequential(*[
            nn.Linear(DIM*4+4,HIDDEN,bias=False),
            nn.BatchNorm1d(HIDDEN),
            nn.ReLU(),
            nn.Linear(HIDDEN,HIDDEN,bias=False),
            nn.BatchNorm1d(HIDDEN),
            nn.ReLU(),
            nn.Linear(HIDDEN,1,bias=False),
            nn.Sigmoid(),
        ])
    def forward(self,x):
        p,s,e,a,conti = x[:,:1].long(),x[:,1:2].long(),x[:,2:3].long(),x[:,3:4].long(),x[:,4:].float()
        x_ = torch.cat([self.emb_p(p).squeeze(1),
                        self.emb_s(s).squeeze(1),
                        self.emb_e(e).squeeze(1),
                        self.emb_a(a).squeeze(1),
                        conti],dim=1)
        return self.mlp(x_)

In [21]:
from ray.matchbox import DF_Dataset,Trainer

In [22]:
train_ds = DF_Dataset(train_df,x_prepro=x_pre.prepro,y_prepro=y_pre.prepro,bs=64,shuffle=False)
valid_ds = DF_Dataset(test_df,x_prepro=x_pre.prepro,y_prepro=y_pre.prepro,bs=64,shuffle=False)

In [23]:
from torch.optim import Adam

In [24]:
model = ti_mlp()

In [25]:
opt = Adam(model.parameters())
loss_func = nn.BCELoss()

In [26]:
def action(*args,**kwargs):
    x,y = args[0]
    x = x.squeeze(0)
    y = y.float()
    opt.zero_grad()
    y_ = model(x)
    loss = loss_func(y_,y)
    acc = accuracy(y_,y.long())
    rec = recall(y_,y.long())
    prec = precision(y_,y.long().squeeze(0))
    f1 = (rec*prec)/(rec+prec)
    loss.backward()
    opt.step()
    return {"loss":loss.item(),"acc":acc,"rec":rec,"prec":prec,"f1":f1}

def val_action(*args,**kwargs):
    x,y = args[0]
    x = x.squeeze(0)
    y = y.float()
    y_ = model(x)
    loss = loss_func(y_,y)
    acc = accuracy(y_,y.long())
    rec = recall(y_,y.long())
    prec = precision(y_,y.long().squeeze(0))
    f1 = (rec*prec)/(rec+prec)
    return {"loss":loss.item(),"acc":acc,"rec":rec,"prec":prec,"f1":f1}

from ray.matchbox import accuracy

def recall(y_pred,y_true):
    is_right = (((y_pred > BENCH_MARK)*1).long()==y_true)
    is_right_in_targ = is_right[y_true==1]
    if len(is_right_in_targ) == 0:
        is_right_in_targ = torch.zeros(1)
    return is_right_in_targ.float().mean()

def precision(y_pred,y_true):
    is_right = (((y_pred > BENCH_MARK)*1).long()==y_true)
    is_right_in_pred = is_right[(y_pred > BENCH_MARK)]
    if len(is_right_in_pred) == 0:
        is_right_in_pred = torch.zeros(1)
    return is_right_in_pred.float().mean()

def f1(recall,precision):
    """
    stantard f1 for binary classification
    recall:
    precision:
    """
    return 2*precision*recall/(precision + recall)

In [27]:
trainer=Trainer(train_ds,batch_size=1,print_on=2)

In [28]:
trainer.action = action
trainer.val_action = val_action

In [29]:
trainer.train(50)

  0%|          | 0/14 [00:00<?, ?it/s]/Users/salvor/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([1, 64, 1])) that is different to the input size (torch.Size([64, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
⭐[ep_0_i_7]	acc	0.648✨	f1	0.331✨	loss	0.518✨	prec	0.745✨	rec	0.597:  36%|███▌      | 5/14 [00:00<00:00, 27.75it/s]/Users/salvor/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([1, 59, 1])) that is different to the input size (torch.Size([59, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
⭐[ep_0_i_13]	acc	0.609✨	f1	0.351✨	loss	0.434✨	prec	0.901✨	rec	0.576: 100%|██████████| 14/14 [00:00<00:00, 45.49it/s]
⭐[ep_1_i_13]	acc	0.648✨	f1	0.310✨	loss	0.489✨	prec	0.767✨	rec	0.532: 